<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_MultiCast_v4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee
All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the
LICENSE file in the root directory of this source tree. 

#Create Charts and store in MongoDB

#Setup

##Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [ ]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python_string_utils
#!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
#!pip install pyswisseph==2.10.2.0.dev1
!apt install mongodb > /dev/null

     |████████████████████████████████| 521 kB 26.6 MB/s 




In [ ]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions



In [ ]:
# ---------------------------
import p21
import p21swe
import p21utils
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21swe)
importlib.reload(p21utils)

imported p21
p21swe imported
p21utils imported
imported p21
p21swe imported
p21utils imported


<module 'p21utils' from '/content/p21utils.py'>

In [ ]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

In [ ]:
#!cat p21utils.py

##Connect to MongoDB

In [ ]:
from pymongo import MongoClient
import pandas as pd
import dateutil
import string_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
# -----------------------------------------------

from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI

# -----------------------------------------------
mongoURI = rMongoURI                                   # for remote server
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

Mounted at /content/drive


['btncgfi3a5bxyea']

In [ ]:
db = client.btncgfi3a5bxyea
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
pd.DataFrame(list(db.list_collections()))
# empty list '[]' means that there are not collections in database

,name,type,options,info,idIndex
0,khona21,collection,{},"{'readOnly': False, 'uuid': b'\xfb\x85\xe1?DR@...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'n..."


In [ ]:
db = client.btncgfi3a5bxyea
#db.restaurants.drop()
#db.khawnaa42.drop()
#db.khona21.drop()
db.create_collection('khona21')      # https://feminisminindia.com/2019/06/07/khawna-khana-poet-rural-bengal/
                                        # https://en.wikipedia.org/wiki/Khana_(poet)
kollection = db.khona21                   # Set the collection to work with

#Check and record Initial DB status
countInitial = kollection.count_documents({})
dbSizeInitial = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(round(dbSizeInitial),'KB database with ', countInitial, ' docs in current collection' )

0 KB database with  0  docs in current collection


##Read External Data
from CSV files


In [ ]:
#import pandas as pd

#!wget -O peopleData.csv -q https://raw.githubusercontent.com/prithwis/parashar21/main/data/Test5Data.txt # 5 Person Test Data

#!gdown -O peopleData.csv https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L # 5 Person Test Data
!gdown -O peopleData.csv https://drive.google.com/uc?id=1dlvqcygJJh0CfyY_X2MLrWm-W9U0z_6l # 39663 Rodden Data
#!sed -i '1d' peopleData.csv 
#!cat peopleData.csv
df = pd.read_csv('peopleData.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1dlvqcygJJh0CfyY_X2MLrWm-W9U0z_6l
To: /content/peopleData.csv
100% 6.33M/6.33M [00:00<00:00, 55.7MB/s]


,Gender,Voc1_Cat,Voc1_Det,Voc2_Cat,Voc2_Det,Voc3_Cat,Voc3_Det,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Entertain/Music,Instrumentalist (Guitarist),Entertain/Music,Song writer,Entertain/Music,"Vocalist/ Pop, Rock, etc.",6,10,1968,17:35,h1e,1.0,48.550000,3.300000,MET,standard,A_Dominique
1,M,Writers,Columnist/ journalist,Writers,Detective/ Mystery,Writers,Fiction (Novelist),19,12,1947,20:00,h1e,1.0,47.383333,0.683333,MET,standard,A_D_G_
2,M,Entertainment,Actor/ Actress,nil,nil,nil,nil,21,2,1911,03:00,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Andreas
3,M,Entertainment,Actor/ Actress,Entertainment,Voice-Over,nil,nil,25,4,1902,07:15,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Per
4,M,Entertainment,TV host/ Personality,Politics,Government employee (Minister),nil,nil,4,1,1933,12:30,h0w,-0.0,38.350000,-0.483333,GMT,standard,Aal_Pomares_Henri_Francois


#Cast & Store chart in Mongo
Driver Code

In [ ]:
p21.ChartType = 'Rashi'

for person in range(len(df)):
    try :
        personData =df.iloc[person]
        p21swe.C02_parsePersonData(personData)            # split the data, extract required data into fields
                                                            # create skeleton chart json doc as p21.chart
        p21swe.C03_convertDates()                         # convert date, time of birth into universal time, calculate Ayanamsha
        p21swe.C04_calculateGrahaPositions()              # calculate the Lon of each Graha along with Retrograde staus

                    #Horoscope is defined in terms of Natal Longitudes and Retrogrades
                    # for example
                    #GLon = {"La":98.5,"Su":178.9,"Mo":250.6,"Ma":196.2,"Me":193.2,"Ju":274.8,"Ve":153.8,"Sa":270.2,"Ra":122.1,"Ke":302.1}
                    #GRet = {"La":False,"Su":False,"Mo":False,"Ma":False,"Me":True,"Ju":False,"Ve":False,"Sa":False,"Ra":True,"Ke":True}

        p21swe.C05_buildGLonGRet()                              # create two json docs ( python dicts) to store Graha Lon and Ret status
                                                                # these are stored as global variables p21.GLon, p21.GRet
        p21utils.appendDict(p21.chart,p21.GLonRet)              # add Lon, Ret information to chart
        
        # ----------------------------------------
        p21utils.R11_LocateGrahaInRashi()
        
        p21utils.C10_DetermineBhavs()                           # determine Houses for person
        p21utils.appendDict(p21.chart,p21.BhavNBhavA)           # add BhavN, BhavA information to chart   
         
        # ----------------------------------------    
        p21utils.C11_DetermineLord()                            # determine Lords of Bhavs
        p21utils.appendDict(p21.chart,p21.LordInfo)             #  add information on Lords   

        p21utils.C12_BhavOfGraha_Lord()
        p21utils.appendDict(p21.chart,p21.BhavOfGraha_LordInfo) #  add information on Bhav Residency of Graha and Lords  
        
        
        p21utils.C21_DeterminePositions()
        p21utils.appendDict(p21.chart,p21.Positions)            #  add information on Graha / Lord POSITIONS  

        p21utils.C31_DetermineAspects()
        p21utils.appendDict(p21.chart,p21.Aspects)              #  add information on Graha / Lord ASPECTS  

        p21utils.C41_DetermineConjuncts()           
        p21utils.appendDict(p21.chart,p21.Conjuncts)              #  add information on Graha / Lord Conjuncts
    except :
        print('error on record ', person)
    else :
        insert_result = kollection.insert_one(p21.chart)        # insert one json doc, chart to database
        insert_result.acknowledged                              # Confirms that insert is successful
    finally:
        if ( person % 1000 ) == 0:
            print(person, "records processed")
    


0 records processed
1000 records processed
2000 records processed
3000 records processed
4000 records processed
5000 records processed
6000 records processed
7000 records processed
8000 records processed
9000 records processed
10000 records processed
11000 records processed
12000 records processed
13000 records processed
14000 records processed
15000 records processed
16000 records processed
17000 records processed
18000 records processed
19000 records processed
20000 records processed
21000 records processed
22000 records processed
23000 records processed
24000 records processed
25000 records processed
26000 records processed
27000 records processed
28000 records processed
error on record  28803
29000 records processed
30000 records processed
31000 records processed
32000 records processed
33000 records processed
34000 records processed
35000 records processed
36000 records processed
37000 records processed
38000 records processed
39000 records processed


#Check Database Status

In [ ]:
countFinal = kollection.count_documents({})
dbSizeFinal = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
DocsInserted = countFinal - countInitial
AddSpaceUsed = round(dbSizeFinal - dbSizeInitial)
if DocsInserted > 0:
    AvgDocSize = round(AddSpaceUsed/DocsInserted)
else:
    AvgDocSize = 'na'
print(DocsInserted,' docs inserted, ',AddSpaceUsed,' KB added , with avg doc size = ', AvgDocSize)
print(round(dbSizeFinal),'KB database with ', countFinal, ' docs in current collection' )

39667  docs inserted,  126831  KB added , with avg doc size =  3
126831 KB database with  39667  docs in current collection


In [ ]:
# Check data loaded into MongoDb
#c = kollection.find()
#for crnt in c:
#    print(crnt)
pd.DataFrame(list(kollection.find({},{'_id':0}).limit(10)))

,pid,GLon,GRet,BhavN,BhavA,Lord,LordRashiN,LordRashiA,GrahaLordBhav,GRashiN,GrahaBhava,LordBhav,exaltG,debilG,ownHouseG,inFriendG,inEnemyG,exaltL,debilL,ownHouseL,inFriendL,inEnemyL,GAspects2,GAspectedBy2,BAspectedBy2,GConjunctsG2,BLConjunctsG2,BLConjunctsBL2
0,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]","[ , Can, Leo, Vir, Lib, Sco, Sag, Cap, Acq, Pi...","[ , Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve...","[ , 9, 6, 7, 6, 7, 10, 10, 10, 10, 7, 6, 7]","[ , Sag, Vir, Lib, Vir, Lib, Cap, Cap, Cap, Ca...","{'Su': [2], 'Mo': [1], 'Ma': [5, 10], 'Me': [3...","{'La': 4, 'Su': 6, 'Mo': 9, 'Ma': 7, 'Me': 7, ...","{'La': 1, 'Su': 3, 'Mo': 6, 'Ma': 4, 'Me': 4, ...","[ , 6, 3, 4, 3, 4, 7, 7, 7, 7, 4, 3, 4]","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, True, False, True...","[False, False, False, False, False, False, Fal...","[False, False, False, True, True, False, False...","[False, False, False, False, False, False, Fal...","{'La': ['Sa', 'Ju'], 'Ma': ['Sa', 'Ju'], 'Ju':...","{'Ju': ['Ma', 'La'], 'Sa': ['Ma', 'La'], 'La':...","{'7': ['Ma', 'La'], '9': ['Su', 'Sa', 'Ve'], '...","{'Su': ['Ve'], 'Ve': ['Su'], 'Ma': ['Me'], 'Me...","{'2': ['Ve'], '3': ['Ma'], '4': ['Su'], '5': [...","{'2': ['4', '11'], '3': ['5', '10'], '4': ['2'..."
1,"{'tags': ['Author', 'Homemaker', 'PostGraduate...","{'La': 310.415, 'Su': 243.701, 'Mo': 201.001, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[ , Acq, Pis, Ari, Tau, Gem, Can, Leo, Vir, Li...","[ , Sa, Ju, Ma, Ve, Me, Mo, Su, Me, Ve, Ma, Ju...","[ , 11, 3, 10, 10, 8, 7, 9, 8, 10, 10, 3, 11]","[ , Acq, Gem, Cap, Cap, Sco, Lib, Sag, Sco, Ca...","{'Su': [7], 'Mo': [6], 'Ma': [3, 10], 'Me': [5...","{'La': 11, 'Su': 9, 'Mo': 7, 'Ma': 10, 'Me': 8...","{'La': 1, 'Su': 11, 'Mo': 9, 'Ma': 12, 'Me': 1...","[ , 1, 5, 12, 12, 10, 9, 11, 10, 12, 12, 5, 1]","{'Su': False, 'Mo': False, 'Ma': True, 'Me': F...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': True, 'Mo': False, 'Ma': False, 'Me': F...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","[False, False, False, True, False, False, Fals...","[False, False, False, False, False, False, Fal...","[False, True, False, False, False, False, Fals...","[False, True, False, False, False, False, Fals...","[False, False, True, False, False, False, Fals...","{'Su': ['Ju'], 'Me': ['Ra'], 'Ju': ['Su', 'Sa'...","{'Ju': ['Su'], 'Ra': ['Me'], 'La': ['Ju'], 'Su...","{'7': ['Sa', 'Ma', 'La'], '5': ['Su'], '3': ['...","{'Ma': ['Ve'], 'Ve': ['Ma'], 'Me': ['Ke'], 'Ke...","{'3': ['Ve'], '4': ['Ma'], '5': ['Ke'], '8': [...","{'3': ['9', '4'], '4': ['10', '3'], '9': ['10'..."
2,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 85.351, 'Su': 172.44, 'Mo': 63.984, 'Ma...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Gem, Can, Leo, Vir, Lib, Sco, Sag, Cap, Ac...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 6, 3, 6, 6, 6, 2, 4, 9, 9, 4, 2, 6]","[ , Vir, Gem, Vir, Vir, Vir, Tau, Can, Sag, Sa...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1...","{'La': 3, 'Su': 6, 'Mo': 3, 'Ma': 2, 'Me': 6, ...","{'La': 1, 'Su': 4, 'Mo': 1, 'Ma': 12, 'Me': 4,...","[ , 4, 1, 4, 4, 4, 12, 2, 7, 7, 2, 12, 4]","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...","{'Su': False, 'Mo': True, 'Ma': False, 'Me': F...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ...",

In [ ]:
#Retrieves Charts based on tags
#pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})))
#c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})
#for crnt in c:
#    print(crnt)

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)